In [3]:
from NID.datasets import Pinky40
from NID.models import DnCNN
from NID.utils import weights_init_kaiming, batch_PSNR

import numpy as np
import scipy.io as sio 
import matplotlib.pyplot as plt 
from mpl_toolkits.axes_grid1 import make_axes_locatable
from dataclasses import dataclass

import os
import h5py
import random
import cv2
import torch 
import torch.nn as nn 
import torch.optim as optim
import torchvision.utils as utils
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from tensorboardX import SummaryWriter 

%matplotlib inline
%reload_ext autoreload
%autoreload 2

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

@dataclass
class training_configurations: 
    '''class for storing options of the netowrk training'''
    preprocess: bool = False  
    batchSize: int = 256    # training batch size 
    num_of_layers: int = 10 # number of total layers 
    epochs: int = 3   #number of training epochs 
    milestone: int = 1 # when to decay learning rate 
    lr: float = 1e-3  # initial learning rate 
    outf: str = 'logs' #path of log files 
    mode: str = 'M' #with known noise level (S) or blind training (B)
    noiseL: float = 0.1 # noise level
    val_noiseL: float = 0.1 # nosie level used on validation set 

# prepare data

In [4]:
dataset_train = Pinky40(train=True)
print(type(dataset_train))
dataset_val = Pinky40(train=False)
loader_train = DataLoader(dataset=dataset_train, num_workers=4, 
                         batch_size=256, shuffle=True)
print("# of training samples: %d\n" % len(dataset_train))
print("# of testing samples: %d\n" % len(dataset_val))

<class 'NID.datasets.Pinky40'>
# of training samples: 40

# of testing samples: 311



# configure training options

In [5]:
opt = training_configurations()
print(type(opt))
testOpt = opt.outf
print(testOpt)

<class '__main__.training_configurations'>
logs


# build model 

In [6]:
net = DnCNN(channels=1, num_of_layers=opt.num_of_layers)
# Move to GPU
device_ids = [0]
#initialize weights
file_results = os.path.join(opt.outf, 'net_{}_0point1NewDataS.pth'.format(opt.num_of_layers))
if os.path.exists(file_results):
    print("we entered the if")
    print('use the trained model as the initialization')
    model = nn.DataParallel(net, device_ids=device_ids).cuda()
    model.load_state_dict(torch.load(file_results))
    model.eval()
else:
    print("we entered the else")
    net.apply(weights_init_kaiming)
    print("we are now doing model = nn.DataParallel...")
    model = nn.DataParallel(net, device_ids=device_ids).cuda()
    print("we exit the else")
    
criterion = nn.MSELoss(reduction='sum')
criterion.cuda()
# Optimizer
optimizer = optim.Adam(model.parameters(), lr=opt.lr)
# training
writer = SummaryWriter(opt.outf)
step = 0
opt.lr = 1e-4

we entered the else
we are now doing model = nn.DataParallel...
we exit the else


# start training - GPU

In [6]:
for epoch in range(opt.epochs):
        if epoch < opt.milestone:
            current_lr = opt.lr
        else:
            current_lr = opt.lr / 10.
        # set learning rate
        for param_group in optimizer.param_groups:
            param_group["lr"] = current_lr
        print('learning rate %f' % current_lr)
        # train
        for i, data in enumerate(loader_train, 0):
            # training step
            model.train()
            model.zero_grad()
            optimizer.zero_grad()
            img_train = data
            if opt.mode == 'S':
                noise = torch.FloatTensor(img_train.size()).normal_(mean=0, std=opt.noiseL)
            elif opt.mode == 'B':
                noise = torch.zeros(img_train.size())
                stdN = np.random.uniform(noiseL_B[0], noiseL_B[1], size=noise.size()[0])
                for n in range(noise.size()[0]):
                    sizeN = noise[0,:,:,:].size()
                    noise[n,:,:,:] = torch.FloatTensor(sizeN).normal_(mean=0, std=stdN[n])
            elif opt.mode = 'M': 
                # delete some pixels 
                noise = torch.FloatTensor(img_train.size()).normal_(mean=0, std=opt.noiseL)

            imgn_train = img_train + noise
            img_train, imgn_train = Variable(img_train.cuda()), Variable(imgn_train.cuda())
            noise = Variable(noise.cuda())
            out_train = model(imgn_train)
            loss = criterion(out_train, noise) / (imgn_train.size()[0]*2)
            loss.backward()
            optimizer.step()
            # results
            model.eval()
            out_train = torch.clamp(imgn_train-model(imgn_train), 0., 1.)
            psnr_train = batch_PSNR(out_train, img_train, 1.)
            print("[epoch %d][%d/%d] loss: %.4f PSNR_train: %.4f" %
                (epoch+1, i+1, len(loader_train), loss.item(), psnr_train))
            # if you are using older version of PyTorch, you may need to change loss.item() to loss.data[0]
            if step % 10 == 0:
                # Log the scalar values
                writer.add_scalar('loss', loss.item(), step)
                writer.add_scalar('PSNR on training data', psnr_train, step)
            step += 1
        ## the end of each epoch
        model.eval()
        # validate
        psnr_val = 0
        for k in range(len(dataset_val)):
            img_val = torch.unsqueeze(dataset_val[k], 0)
            if opt.mode == 'S':
                noise = torch.FloatTensor(img_val.size()).normal_(mean=0, std=opt.noiseL)
            elif opt.mode == 'B':
                noise = torch.zeros(img.size())
                stdN = np.random.uniform(noiseL_B[0], noiseL_B[1], size=noise.size()[0])
                for n in range(noise.size()[0]):
                    sizeN = noise[0,:,:,:].size()
                    noise[n,:,:,:] = torch.FloatTensor(sizeN).normal_(mean=0, std=stdN[n])
            elif opt.mode = 'M': 
                # delete some pixels 
                noise = torch.FloatTensor(img_val.size()).normal_(mean=0, std=opt.noiseL)

            noise = torch.FloatTensor(img_val.size()).normal_(mean=0, std=opt.val_noiseL)
            imgn_val = img_val + noise
            img_val, imgn_val = Variable(img_val.cuda()), Variable(imgn_val.cuda())
            out_val = torch.clamp(imgn_val-model(imgn_val), 0., 1.)
            psnr_val += batch_PSNR(out_val, img_val, 1.)
        psnr_val /= len(dataset_val)
        print("\n[epoch %d] PSNR_val: %.4f" % (epoch+1, psnr_val))
        writer.add_scalar('PSNR on validation data', psnr_val, epoch)
        # log the images
#         out_train = torch.clamp(imgn_train-model(imgn_train), 0., 1.)
#         Img = utils.make_grid(img_train.data, nrow=8, normalize=True, scale_each=True)
#         Imgn = utils.make_grid(imgn_train.data, nrow=8, normalize=True, scale_each=True)
#         Irecon = utils.make_grid(out_train.data, nrow=8, normalize=True, scale_each=True)
#         writer.add_image('clean image', Img, epoch)
#         writer.add_image('noisy image', Imgn, epoch)
#         writer.add_image('reconstructed image', Irecon, epoch)
        # save model
        torch.save(model.state_dict(), os.path.join(opt.outf, 'net_{}.pth'.format(opt.num_of_layers)))


learning rate 0.000100
[epoch 1][1/834] loss: 32.3205 PSNR_train: 16.8717
[epoch 1][2/834] loss: 31.8563 PSNR_train: 16.8466
[epoch 1][3/834] loss: 31.5392 PSNR_train: 16.8560
[epoch 1][4/834] loss: 31.0924 PSNR_train: 16.8602
[epoch 1][5/834] loss: 30.7566 PSNR_train: 16.8688
[epoch 1][6/834] loss: 30.4676 PSNR_train: 16.8353
[epoch 1][7/834] loss: 30.0385 PSNR_train: 16.8430
[epoch 1][8/834] loss: 29.7016 PSNR_train: 16.8309
[epoch 1][9/834] loss: 29.3083 PSNR_train: 16.8469
[epoch 1][10/834] loss: 28.8119 PSNR_train: 16.8552
[epoch 1][11/834] loss: 28.4581 PSNR_train: 16.8576
[epoch 1][12/834] loss: 28.0682 PSNR_train: 16.8399
[epoch 1][13/834] loss: 27.5922 PSNR_train: 16.8511
[epoch 1][14/834] loss: 27.2345 PSNR_train: 16.8366
[epoch 1][15/834] loss: 26.7839 PSNR_train: 16.8336
[epoch 1][16/834] loss: 26.3449 PSNR_train: 16.8388
[epoch 1][17/834] loss: 25.8914 PSNR_train: 16.8185
[epoch 1][18/834] loss: 25.3613 PSNR_train: 16.8288
[epoch 1][19/834] loss: 24.9583 PSNR_train: 16.814